<a href="https://colab.research.google.com/github/linneasandersen/dalas-project/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   Get data
- Wikipedia
- OEC
- FAO
2.   Store data in pandas
3. Inspect




In [1]:
import requests
import pandas as pd
from pathlib import Path

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# OEC

In [24]:
# Define the years to retrieve data for
years = range(1995, 2024)

In [22]:
# Iterate and fetch export data
df_exports = []
df_imports = []
url = "https://api-v2.oec.world/tesseract/data.jsonrecords"

for year in years:
    params_export = {
        "cube": "trade_i_baci_a_92",
        "locale": "en",
        "drilldowns": "Exporter Country,Year,HS4",
        "measures": "Trade Value",
        "include": f"Year:{year}"
    }
    r = requests.get(url, params=params_export)
    try:
        df = pd.DataFrame(r.json()["data"])
        df_exports.append(df)
    except KeyError:
        print(f"No data found for export in {year}")

    params_import = {
    "cube": "trade_i_baci_a_92",
    "locale": "en",
    "drilldowns": "Importer Country,Year,HS4",
    "measures": "Trade Value",
    "include": f"Year:{year}"
    }
    r = requests.get(url, params=params_import)
    try:
        df = pd.DataFrame(r.json()["data"])
        df_imports.append(df)
    except KeyError:
        print(f"No data found for import in {year}")


# concatenate
df_exports = pd.concat(df_exports)
df_imports = pd.concat(df_imports)

print("Finished fetching export data.")

Finished fetching export data.


In [23]:
# display data
display(df_exports)
display(df_imports)
print(df_exports.columns)
print(df_imports.columns)


# more possibilities for columns, see cubeOECinfo.json

,Exporter Country ID,Exporter Country,HS4 ID,HS4,Year,Trade Value
0,afago,Angola,10302,Non-fillet Fresh Fish,1995,35266.0
1,afago,Angola,10303,Non-fillet Frozen Fish,1995,7956102.0
2,afago,Angola,10304,Fish Fillets,1995,278508.0
3,afago,Angola,10305,Dried/Salted/Smoked/Brined Fish,1995,3682.0
4,afago,Angola,10306,Crustaceans,1995,21217946.0
...,...,...,...,...,...,...
175624,saven,Venezuela,219702,Prints,2023,1327961.0
175625,saven,Venezuela,219703,Sculptures,2023,8853599.0
175626,saven,Venezuela,219704,Revenue Stamps,2023,147449.0
175627,saven,Venezuela,219705,Collector's Items,2023,158160.0


,HS4 ID,HS4,Importer Country ID,Importer Country,Year,Trade Value
0,10101,Horses,afbfa,Burkina Faso,1995,1351.0
1,10101,Horses,afciv,Cote d'Ivoire,1995,9136.0
2,10101,Horses,afcod,Democratic Republic of the Congo,1995,1811.0
3,10101,Horses,afcpv,Cape Verde,1995,567.0
4,10101,Horses,afdza,Algeria,1995,30570.0
...,...,...,...,...,...,...
233276,219706,Antiques,saecu,Ecuador,2023,3894.0
233277,219706,Antiques,saguy,Guyana,2023,14955.0
233278,219706,Antiques,saper,Peru,2023,335766.0
233279,219706,Antiques,sapry,Paraguay,2023,61105.0


Index(['Exporter Country ID', 'Exporter Country', 'HS4 ID', 'HS4', 'Year',
       'Trade Value'],
      dtype='object')
Index(['HS4 ID', 'HS4', 'Importer Country ID', 'Importer Country', 'Year',
       'Trade Value'],
      dtype='object')


Data set size is order of magnitude 10^6 for both import and export
Currently 6 columns, but more possibilities

## Harmonized system of products

In [25]:
data_dir = Path("/content/drive/MyDrive/DALAS/data/raw")
data_path_products = data_dir / "hs_product_classification.csv"

In [26]:
print(data_path_products)
df = pd.read_csv(
    data_path_products,
    sep=';',        # semicolon separator
    quotechar='"'
)
print(df.head())
print(df.columns)

/content/drive/MyDrive/DALAS/data/raw/hs_product_classification.csv
    OEC ID     HS ID Product Level  \
0        1         1       Section   
1      101        01           HS2   
2    10101     01.01           HS4   
3  1010110  01.01.10           HS6   
4  1010111  01.01.11           HS6   

                                        Product Name  \
0                                    Animal Products   
1                                       Live animals   
2                                             Horses   
3  Live horses/asses/mules/hinnies: pure-bred bre...   
4                          Pure-bred Breeding Horses   

                             Revision  
0  HS92,HS96,HS02,HS07,HS12,HS17,HS22  
1  HS92,HS96,HS02,HS07,HS12,HS17,HS22  
2  HS92,HS96,HS02,HS07,HS12,HS17,HS22  
3                           HS02,HS07  
4                           HS92,HS96  
Index(['OEC ID', 'HS ID', 'Product Level', 'Product Name', 'Revision'], dtype='object')


# FAO


In [27]:
data_dir = Path("/content/drive/MyDrive/DALAS/data/raw")
zip_file = data_dir / "Environment_Temperature_change_E_All_Data.zip"

In [28]:
from pathlib import Path
from zipfile import ZipFile

extract_dir = data_dir / "extracted"
extract_dir.mkdir(parents=True, exist_ok=True)

# Extract all files
with ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to {extract_dir}")


Files extracted to /content/drive/MyDrive/DALAS/data/raw/extracted


In [32]:
# Find extracted Excel files
files = list(extract_dir.glob("*.csv"))

for file in files:
    print(file.name)

Environment_Temperature_change_E_All_Data.csv
Environment_Temperature_change_E_All_Data_NOFLAG.csv
Environment_Temperature_change_E_AreaCodes.csv
Environment_Temperature_change_E_Elements.csv
Environment_Temperature_change_E_Flags.csv


In [40]:
# print columns of all datasets, size of table
for file in files:
    print(file.name)
    df = pd.read_csv(file)
    print(df.shape)
    print(df.columns)
    print("---------------------------------------------------------------------------------")


Environment_Temperature_change_E_All_Data.csv
(9656, 136)
Index(['Area Code', 'Area Code (M49)', 'Area', 'Months Code', 'Months',
       'Element Code', 'Element', 'Unit', 'Y1961', 'Y1961F',
       ...
       'Y2020', 'Y2020F', 'Y2021', 'Y2021F', 'Y2022', 'Y2022F', 'Y2023',
       'Y2023F', 'Y2024', 'Y2024F'],
      dtype='object', length=136)
---------------------------------------------------------------------------------
Environment_Temperature_change_E_All_Data_NOFLAG.csv
(9656, 72)
Index(['Area Code', 'Area Code (M49)', 'Area', 'Months Code', 'Months',
       'Element Code', 'Element', 'Unit', 'Y1961', 'Y1962', 'Y1963', 'Y1964',
       'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972',
       'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980',
       'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987', 'Y1988',
       'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996',
       'Y1997', 'Y1998', 'Y1999', 'Y2000',

In [41]:
# Load first file
file = files[0]
print(file)
df = pd.read_csv(file)
print(df.head())

/content/drive/MyDrive/DALAS/data/raw/extracted/Environment_Temperature_change_E_All_Data.csv
   Area Code Area Code (M49)         Area  Months Code    Months  \
0          2            '004  Afghanistan         7001   January   
1          2            '004  Afghanistan         7001   January   
2          2            '004  Afghanistan         7002  February   
3          2            '004  Afghanistan         7002  February   
4          2            '004  Afghanistan         7003     March   

   Element Code             Element Unit  Y1961 Y1961F  ...  Y2020 Y2020F  \
0          7271  Temperature change   °c  0.792      E  ... -0.622      E   
1          6078  Standard Deviation   °c  1.964      E  ...  1.964      E   
2          7271  Temperature change   °c -1.750      E  ...  2.565      E   
3          6078  Standard Deviation   °c  2.606      E  ...  2.606      E   
4          7271  Temperature change   °c  0.483      E  ...  0.647      E   

   Y2021 Y2021F  Y2022 Y2022F  Y20

9656 rows, 136 columns

# Wikipedia

